In [1]:
! nvidia-smi

Thu Apr 24 00:24:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:01:00.0  On |                  N/A |
| 31%   43C    P0             44W /  170W |    2329MiB /  12288MiB |      6%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
os.environ['CUDA_VISABLE_DEVICES']='0'

In [3]:
#! pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121

In [4]:
import torch
print(torch.cuda.is_available())

True


官网地址： https://download.pytorch.org/whl/torch_stable.html

# 1. modelscope调用qwen2 1.5b模型

In [5]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('Qwen/Qwen2.5-1.5B-Instruct',local_dir='../model/Qwen-1.5-Instruct')

2025-04-24 00:24:38,667 - modelscope - INFO - Target directory already exists, skipping creation.


In [6]:
from modelscope import AutoModelForCausalLM, AutoTokenizer,GenerationConfig
import torch
model_dir =  '../model/Qwen-1.5-Instruct'

model = AutoModelForCausalLM.from_pretrained(model_dir, torch_dtype="auto",
    device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_dir)
gen_config = GenerationConfig.from_pretrained(model_dir)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [7]:
model.device

device(type='cuda', index=0)

In [8]:
gen_config

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [9]:
tokenizer("你好呀！")

{'input_ids': [108386, 104256, 6313], 'attention_mask': [1, 1, 1]}

In [21]:
def run_prompt(prompt,temperature=0.1,top_k=20,top_p=0.8,max_new_tokens =2048):
    gen_config.temperature =temperature
    gen_config.top_p=top_p
    gen_config.top_k=top_k
    
    messages = [
    {"role": "system", "content": "你是乐于助人的助手."},
    {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
    )
    print(text)
    model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
    
    generated_ids = model.generate(
    **model_inputs,
    # model_inputs.inputs_ids,
    max_new_tokens= max_new_tokens,
    generation_config = gen_config
    )
    generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [25]:
prompt = 'hello'
run_prompt(prompt)

<|im_start|>system
你是乐于助人的助手.<|im_end|>
<|im_start|>user
hello<|im_end|>
<|im_start|>assistant



'你好！有什么我可以帮助你的吗？'

# 2 huggingface transformers 调用本地开源模型chatglm3

In [38]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('ZhipuAI/chatglm3-6b',local_dir='../model/chatglm3-6b',ignore_patterns='*.bin')# ,ignore_file_pattern='*.bin'

2025-04-24 09:47:33,586 - modelscope - WARNING - Model revision not specified, use revision: v1.0.2


In [1]:
from transformers import AutoModel,AutoTokenizer

In [2]:
model_path =  '../model/chatglm3-6b'

In [4]:
import torch
tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code = True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True, low_cpu_mem_usage=True, torch_dtype=torch.half, device_map="auto")

e:\program\anaconda\envs\llamaIndex\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
e:\program\anaconda\envs\llamaIndex\lib\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [5]:
# 推理函数
def chatglm(prompt,history=[],model = model,tokenizer=tokenizer):
    response,history = model.chat(tokenizer,
                                  prompt,
                                  history=[],
                                  temperature=0.1,top_p=0.8,top_k=20,
                                  max_length = 8192)
    return response

In [6]:
chatglm("介绍你自己")

C:\Users\Yun/.cache\huggingface\modules\transformers_modules\chatglm3-6b\modeling_chatglm.py:226: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(query_layer, key_layer, value_layer,


'你好，我是 ChatGLM3-6B，是清华大学KEG实验室和智谱AI公司共同训练的语言模型。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计算机程序，所以我没有自我意识，也不能像人类一样感知世界。我只能通过分析我所学到的信息来回答问题。'

In [7]:
## 流式输出
def chatglm_stream(prompt,history=[],model = model,tokenizer=tokenizer):
    for data in model.stream_chat(tokenizer,
                                  prompt,
                                  history=[],
                                  temperature=0.1,top_p=0.8,top_k=20,
                                  max_length = 8192):
        yield data[0]
    

In [8]:
from IPython.display import clear_output
user_prompt = "你好请介绍一下你自己"

In [9]:
for res in chatglm_stream(user_prompt):
    clear_output(wait=True)
    print(res)

你好，我是 ChatGLM3-6B，是清华大学KEG实验室和智谱AI公司共同训练的语言模型。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计算机程序，所以我没有自我意识，也不能像人类一样感知世界。我只能通过分析我所学到的信息来回答问题。


# 3 langchain封装llm

In [11]:
from langchain.llms.base import LLM
from typing import Any,List,Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from modelscope import AutoModelForCausalLM, AutoTokenizer,GenerationConfig

# 继承LLM类，重写call函数
class ChatGLM(LLM):
    tokenizer:AutoTokenizer = None
    model:AutoModelForCausalLM = None
    def __init__(self,model,tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model
    def _call(self, prompt:str, stop:Optional[List[str]]=None, run_manager:Optional[CallbackManagerForLLMRun]=None, **kwargs:Any):
        response,history = self.model.chat(self.tokenizer,prompt,history=[],temperature = 0.1,top_p=0.8)
        return response

    @property
    def _llm_type(self)->str:
        return "chatglm"
        

In [12]:
chatglm = ChatGLM(model,tokenizer)
chatglm("你好呀！")

C:\Users\Yun\AppData\Local\Temp\ipykernel_38728\2172477371.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chatglm("你好呀！")


'你好👋！我是人工智能助手 ChatGLM3-6B，很高兴见到你，欢迎问我任何问题。'

# 4. 在线调用讯飞星火大模型

略

# 5. 问题
numpy的兼容性问题：pip install numpy==1.26.4

In [ ]:
# Cannot import available module of AutoModelForCausalLM in modelscope, or related packages(['transformers', 'peft', 'diffusers'])
# ! pip install transformers peft diffusers --upgrade

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached peft-0.15.2-py3-none-any.whl.metadata (13 kB)
  Using cached diffusers-0.33.1-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached peft-0.15.2-py3-none-any.whl (411 kB)
Using cached diffusers-0.33.1-py3-none-any.whl (3.6 MB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.29.3
    Uninstalling huggingface-hub-0.29.3:
      Successfully uninstalled huggingface-hub-0.29.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.0
    Uninstalling transformers-4.50.0:
      Successfully uninstalled transformers-4.50.0


缺少pip install sentencepiece包

In [ ]:
# ! pip install sentencepiece

  Using cached sentencepiece-0.2.0-cp310-cp310-win_amd64.whl.metadata (8.3 kB)
Using cached sentencepiece-0.2.0-cp310-cp310-win_amd64.whl (991 kB)


In [ ]:
# ! pip install safetensors

In [ ]:
# ! pip uninstall transformers -y

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3


In [ ]:
# ! pip install transformers==4.33.3 

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cohere 5.14.0 requires tokenizers<1,>=0.15, but you have tokenizers 0.13.3 which is incompatible.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.33.3 which is incompatible.



  Using cached tokenizers-0.13.3-cp310-cp310-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/7.6 MB 6.7 MB/s eta 0:00:02
   ---------------------------------------- 7.6/7.6 MB 23.6 MB/s eta 0:00:00
Using cached tokenizers-0.13.3-cp310-cp310-win_amd64.whl (3.5 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1


In [7]:
! pip install safetensors